### 日志数据处理

In [34]:
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn

In [ ]:
# 读取 Excel 文件
file_path = './data/raw_data.xlsx'  # 替换为你的 Excel 文件路径
sheet_name = 'Sheet1'  # 指定工作表名称，如果不指定，则默认读取第一个工作表

# 使用 pandas 的 read_excel 函数读取数据
data = pd.read_excel(file_path, sheet_name=sheet_name)

In [26]:
# 真实有用的列: APPID, IP_ADDRESS, 创建时间, 患者ID, 状态
# 初筛：患者ID, (APPID, IP_ADDRESS)
# 嵌入：创建时间, 患者ID, 状态
data = data[["订单创建时间", "患者ID", "状态"]]
print(data.columns)

Index(['订单创建时间', '患者ID', '状态'], dtype='object')


#### 数据的一些特征

1. 一共35万条数据，一共28万个用户，说明大部分用户其实都只挂过一次号(时间跨度大概是一个月)
   - 这些只挂过一次号的基本可以认为是正常用户
   - 先找出重复挂号的用户，分析其特征

In [ ]:
# 找出重复挂号的用户的ID
# duplicated_rows = data[data['患者ID'].duplicated(keep=False)]
# duplicated_rows = duplicated_rows.sort_values(by=['患者ID', '订单创建时间'], ascending=[True, True])

In [ ]:
# 保存
# output_file = "./data/repeat_records.xlsx"
# duplicated_rows.to_excel(output_file, sheet_name="duplicate", index=False)

In [27]:
MAX_DUP = 5
# 筛选出重复次数大于MAX_DUP的
value_counts = data['患者ID'].value_counts()
# 筛选出重复次数大于5的行
mass_duplicated = data[data['患者ID'].isin(value_counts[value_counts > MAX_DUP].index)]
mass_duplicated = mass_duplicated.sort_values(by=['患者ID', '订单创建时间'], ascending=[True, True])
print("大量重复的行: ", len(mass_duplicated))

大量重复的行:  22225


In [28]:
# 筛选无重复的行
unique_rows = data[data['患者ID'].isin(value_counts[value_counts == 1].index)]
unique_rows = unique_rows.sort_values(by=['患者ID', '订单创建时间'], ascending=[True, True])
print("无重复的行: ", len(unique_rows))

无重复的行:  154603


In [33]:
# 选择需要One-Hot编码的列
columns_to_encode = ["状态"]

# 对指定列进行One-Hot编码
encoded_data = pd.get_dummies(data, columns=columns_to_encode, drop_first=True)

# 打印结果
print(encoded_data.shape)

(348210, 8)


#### 嵌入

In [ ]:
def layer_embedding(data, seg_name, embedding_dim=64):
    seg_data = data[seg_name]
    unique_num = seg_data.nunique()
    embedding = nn.Embedding(num_embeddings=unique_num, embedding_dim=embedding_dim) # 定义嵌入层
    index = seg_data.astype('category').cat.codes   # 转换为数值索引
    tensor = torch.tensor(index.values)     # 将数值索引转换为PyTorch张量
    embeddings = embedding(tensor)        # 使用嵌入层获取嵌入向量
    return embeddings


In [ ]:
# 患者ID, 时间(使用嵌入层)
user_embeddings = layer_embedding(data, '患者ID', embedding_dim=16)
time_embeddings = layer_embedding(data, '订单创建时间', embedding_dim=32)
print(user_embeddings.shape)
print(time_embeddings.shape)

torch.Size([348210, 16])
torch.Size([348210, 32])


In [ ]:
# 状态(One-hot)
# 对状态列进行One-Hot编码
data_encoded = pd.get_dummies(data, columns=['状态'])
data_encoded = data_encoded[['状态_主动取消', '状态_医保换号', '状态_已挂号', '状态_已退号', '状态_无号退款',
       '状态_窗口退号', '状态_超时取消']]

# 将One-Hot编码结果转换为张量
state_tensor = torch.tensor(data_encoded.values, dtype=torch.float32)
print(state_tensor.shape)

torch.Size([348210, 7])


In [55]:
# 拼接
embed_vec = torch.cat((user_embeddings, time_embeddings, state_tensor), dim=1)
print(embed_vec.shape)

torch.Size([348210, 55])
